## DV2599 Assignment 1
Group 8


***
Viktor Fransson

vifr22@student.bth.se

***

Tobias Gustafsson

togu22@student.bth.se
***

#### Initialize

In [113]:
# Import packages
import pandas as pd
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from imblearn.over_sampling import SMOTE

# Load dataset
df = pd.read_csv('winequality-white.csv', delimiter=";")

#### 1. Inspect the dataset

In [114]:
# Describe
# df.describe()
# df.info()

# Calculate ratios of quality classes
class_counts = df['quality'].value_counts().sort_index()

print("Quality Class Ratios:")
for class_label, count in class_counts.items():
    ratio = count / len(df)
    print(f"Class {class_label}: {ratio:.2%}")


# Calculate ratio of feature values per quality class
# Get all feature columns
feature_columns = [col for col in df.columns if col != 'quality']

# Create copy of df to change and keep original intact
df_discretisized = df.copy()

for feature in feature_columns:
    df_discretisized[feature] = pd.qcut(df[feature], q=3, labels=['low', 'medium', 'high'])

# Analyze each target class
print("\nTarget Class Feature Analysis:")
for class_label in sorted(df_discretisized['quality'].unique()):
    print(f"\nClass {class_label}:")
    
    # Subset df for the current class
    class_df = df_discretisized[df_discretisized['quality'] == class_label]
    
    # Find the most frequent feature value for each feature
    for feature in feature_columns:
        top_values = class_df[feature].value_counts().head(2)
        print(f"  {feature}:")
        for value, count in top_values.items():
            print(f"    {value}: {count/len(class_df):.1%}")

Quality Class Ratios:
Class 3: 0.41%
Class 4: 3.33%
Class 5: 29.75%
Class 6: 44.88%
Class 7: 17.97%
Class 8: 3.57%
Class 9: 0.10%

Target Class Feature Analysis:

Class 3:
  fixed acidity:
    high: 50.0%
    low: 25.0%
  volatile acidity:
    high: 45.0%
    medium: 30.0%
  citric acid:
    high: 40.0%
    low: 30.0%
  residual sugar:
    high: 40.0%
    low: 35.0%
  chlorides:
    low: 40.0%
    high: 35.0%
  free sulfur dioxide:
    low: 45.0%
    high: 30.0%
  total sulfur dioxide:
    high: 55.0%
    low: 35.0%
  density:
    high: 40.0%
    medium: 35.0%
  pH:
    low: 35.0%
    medium: 35.0%
  sulphates:
    low: 45.0%
    high: 30.0%
  alcohol:
    medium: 45.0%
    low: 35.0%

Class 4:
  fixed acidity:
    high: 39.9%
    low: 34.4%
  volatile acidity:
    high: 57.7%
    medium: 27.0%
  citric acid:
    low: 48.5%
    high: 31.9%
  residual sugar:
    low: 48.5%
    medium: 30.1%
  chlorides:
    high: 42.9%
    medium: 30.1%
  free sulfur dioxide:
    low: 69.3%
    medium: 

#### 2. Split into train and test sets

In [115]:
# Set quality as target
y = df['quality']
x = df.drop('quality', axis=1)

# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

#### 3. Scaling

In [116]:
# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform train set
x_train_scaled = scaler.fit_transform(x_train)

# Transform test set
x_test_scaled = scaler.transform(x_test)

# Turn back into pandas dataframes
x_train_scaled = pd.DataFrame(x_train_scaled, columns=x_train.columns, index=x_train.index)
x_test_scaled = pd.DataFrame(x_test_scaled, columns=x_test.columns, index=x_test.index)

# Describe
x_train_scaled.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000,3918.000000
mean,0.294058,0.225301,0.201011,0.088180,0.113775,0.115835,0.299881,0.132590,0.425929,0.313716,0.417294
std,0.081704,0.114878,0.074042,0.077265,0.075120,0.060019,0.098942,0.057846,0.137889,0.133100,0.204526
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.240385,0.146893,0.162651,0.016871,0.079861,0.073171,0.229698,0.088910,0.327273,0.220930,0.247934
50%,0.288462,0.203390,0.186747,0.069018,0.104167,0.111498,0.290023,0.126712,0.418182,0.290698,0.396694
75%,0.336538,0.271186,0.228916,0.140721,0.128472,0.153310,0.366589,0.171842,0.509091,0.383721,0.561983
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### 4. Repeated k-Fold Cross Validation

In [117]:
# Initialize Repeated k-Fold Cross Validator
rkf = RepeatedKFold(n_splits=3, n_repeats=10)

# Random forest classifier and Logistic regression
classifiers = {
    "Random Forest" : RandomForestClassifier(),
    "Decision Tree" : DecisionTreeClassifier()
}

# Perform cross-validation
results = {}
for name, clf in classifiers.items():
    cv_scores = cross_val_score(clf, x_train_scaled, y_train, cv=rkf, scoring="accuracy")

    results[name] = {
        "average" : round(cv_scores.mean(), 4),
        "standard deviation" : round(cv_scores.std(), 4)
    }

results

{'Random Forest': {'average': 0.6455, 'standard deviation': 0.0096},
 'Decision Tree': {'average': 0.5623, 'standard deviation': 0.0103}}

#### 5. Final model

In [118]:
# Random forest was the best classifier
rf_model = RandomForestClassifier()

rf_model.fit(x_train_scaled, y_train)

RandomForestClassifier()

#### 6. Performance on test set

In [119]:
# Run model on test set
y_pred = rf_model.predict(x_test_scaled)

# Check accuracy
accuracy = accuracy_score(y_pred, y_test)
accuracy

0.6836734693877551

#### 7. Balance scaled train set

In [ ]:
# Apply SMOTE to train set to balance
smote = SMOTE(k_neighbors=3) # k_neigbors < smallest class size (4)

x_train_bal, y_train_bal = smote.fit_resample(x_train_scaled, y_train)

# Print out the ratios
balanced_class_counts = pd.Series(y_train_bal).value_counts()

print("Balanced Quality Class Ratios:")
for class_label, count in balanced_class_counts.items():
    ratio = count / len(y_train_bal)
    print(f"Class {class_label}: {ratio:.2%}")

Quality Class Ratios:
Class 6: 14.29%
Class 7: 14.29%
Class 5: 14.29%
Class 4: 14.29%
Class 8: 14.29%
Class 9: 14.29%
Class 3: 14.29%


#### 8. Validation and fitting on balanced set

In [121]:
# Perform cross-validation (may take a while)
results = {}
for name, clf in classifiers.items():
    cv_scores = cross_val_score(clf, x_train_bal, y_train_bal, cv=rkf, scoring="accuracy")

    results[name] = {
        "average" : round(cv_scores.mean(), 4),
        "standard deviation" : round(cv_scores.std(), 4)
    }

results

{'Random Forest': {'average': 0.8932, 'standard deviation': 0.0052},
 'Decision Tree': {'average': 0.8163, 'standard deviation': 0.0057}}

In [122]:
# Random forest was the best classifier again
rf_model_bal = RandomForestClassifier()

rf_model_bal.fit(x_train_bal, y_train_bal)

RandomForestClassifier()

#### 9. Performance of model trained on balanced set

In [123]:
# Run model on test set
y_pred = rf_model_bal.predict(x_test_scaled)

# Check for improvement
accuracy_bal = accuracy_score(y_pred, y_test)
print("Accuracy improvement:", round(accuracy_bal - accuracy, 3))

Accuracy improvement: -0.028
